In [1]:
#%pyspark

from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
#from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType,DecimalType,IntegerType,FloatType,DateType,TimestampType
from pyspark.sql.types import *
from pyspark.sql.functions import explode;
import pyspark;
import os;
from datetime import datetime 
#import pandas as pd;

import json;
import gzip;



In [3]:
import os

os.chdir('/Users/demonxy468/Desktop/Jump/Spark/Data')
os.getcwd()

'/Users/demonxy468/Desktop/Jump/Spark/Data'

In [7]:
#%pyspark
#rdd = sc.textFile("s3://amazon-product-offers/offer-sample/ae-product-offers-5-2017-04-01*.gz")

#offerDF = sqlContext.read.json("s3://amazon-product-offers/offer-sample/ae-product-offers-5-2017-04-01*.gz")
key = '/Users/demonxy468/Desktop/Jump/Spark/Data/ae-product-offers-5-2017-04-12-23-00-01-ef101cde-63a7-45c2-966e-b62ca7a842cb.gz'
#offerDF = sqlContext.read.json("s3://amazon-product-offers/offer-sample/ae-product-offers-5-2017-04-01-01-24-00-db838163-b7a6-42f0-bfb0-76fa8c6bb2bc.gz")

offerDF = sqlContext.read.json(key)

In [17]:
offerDF.show()

+-----------+----------+-------------+--------------------+--------------------+------+
|ConditionId|    ItemId|MarketplaceId|       ProductOffers|   TimeOfOfferChange|UserId|
+-----------+----------+-------------+--------------------+--------------------+------+
|          4|B008JFQ3O0|         6386|[[new,ATVPDKIKX0D...|/Date(14920379303...|  8828|
+-----------+----------+-------------+--------------------+--------------------+------+



In [12]:
%pyspark

res = offerDF.withColumn("ItemCondition", col("ProductOffers.Identifier.ItemCondition"))\
.withColumn("IsBuyBoxWinner", explode(col("ProductOffers.Offers.IsBuyBoxWinner")))\
.withColumn("IsExpeditedShippingAvailable", explode(col("ProductOffers.Offers.IsExpeditedShippingAvailable")))\
.withColumn("IsFeaturedMerchant", explode(col("ProductOffers.Offers.IsFeaturedMerchant")))\
.withColumn("listingprice_amount", explode(col("ProductOffers.Offers.ListingPrice.Amount")))\
.withColumn("listingprice_currency", explode(col("ProductOffers.Offers.ListingPrice.CurrencyCode")))\
.withColumn("Points", explode(col("ProductOffers.Offers.Points")))\
.withColumn("FeedbackCount", explode(col("ProductOffers.Offers.SellerFeedbackRating.FeedbackCount")))\
.withColumn("SellerPositiveFeedbackRating", explode(col("ProductOffers.Offers.SellerFeedbackRating.SellerPositiveFeedbackRating")))\
.withColumn("SellerId", explode(col("ProductOffers.Offers.SellerId")))\
.withColumn("shipping_amount", explode(col("ProductOffers.Offers.Shipping.Amount")))\
.withColumn("shipping_currency", explode(col("ProductOffers.Offers.Shipping.CurrencyCode")))\
.withColumn("availabilityType", explode(col("ProductOffers.Offers.ShippingTime.AvailabilityType")))\
.withColumn("availableDate", explode(col("ProductOffers.Offers.ShippingTime.AvailableDate")))\
.withColumn("maximumHours", explode(col("ProductOffers.Offers.ShippingTime.MaximumHours")))\
.withColumn("minimumHours", explode(col("ProductOffers.Offers.ShippingTime.MinimumHours")))\
.withColumn("shipsDomestically", explode(col("ProductOffers.Offers.ShipsDomestically")))\
.withColumn("shipsFrom", explode(col("ProductOffers.Offers.ShipsFrom")))\
.withColumn("shipsInternationally", explode(col("ProductOffers.Offers.ShipsInternationally")))\
.withColumn("subCondition", explode(col("ProductOffers.Offers.SubCondition")))\
.withColumn("BuyBoxEligibleOffersCondition", explode(col("ProductOffers.Summary.BuyBoxEligibleOffers.Condition")))\
.withColumn("FulfillmentChannel", explode(col("ProductOffers.Summary.BuyBoxEligibleOffers.FulfillmentChannel")))\
.withColumn("BuyBoxEligibleOffersCount", explode(col("ProductOffers.Summary.BuyBoxEligibleOffers.Value")))\
.withColumn("LandedPrice_Amount", explode(col("ProductOffers.Summary.BuyBoxPrices.LandedPrice.Amount")))\
.withColumn("LandedPrice_Currency", explode(col("ProductOffers.Summary.BuyBoxPrices.LandedPrice.CurrencyCode")))\
.withColumn("BB_listPrice_Amount", explode(col("ProductOffers.Summary.BuyBoxPrices.ListingPrice.Amount")))\
.withColumn("BB_ListPrice_Currency", explode(col("ProductOffers.Summary.BuyBoxPrices.ListingPrice.CurrencyCode")))\
.withColumn("Points", explode(col("ProductOffers.Summary.BuyBoxPrices.Points")))\
#.withColumn("ListPrice_Amount", col("ProductOffers.Summary.ListPrice.Amount"))\
#.withColumn("ListPrice_Currency", col("ProductOffers.Summary.ListPrice.CurrencyCode"))\
.withColumn("ListingPrice", explode(col("ProductOffers.Summary.LowestPrices.ListingPrice.Amount")))\
.withColumn("ListingPrice_Currency", explode(col("ProductOffers.Summary.LowestPrices.ListingPrice.CurrencyCode")))\
.withColumn("MinimumAdvertisedPrice", col("ProductOffers.Summary.MinimumAdvertisedPrice"))\
.withColumn("SalesRankings", explode(col("ProductOffers.Summary.SalesRankings.ProductCategoryId")))\
.withColumn("NumberOfOffers", explode(col("ProductOffers.Summary.SalesRankings.Rank")))\
.withColumn("SuggestedLowerPricePlusShipping",col("ProductOffers.Summary.SuggestedLowerPricePlusShipping"))\
.withColumn("TotalOfferCount",col("ProductOffers.Summary.TotalOfferCount"))\
.withColumn('OfOfferChangeDatetime', from_unixtime((regexp_replace('TimeOfOfferChange','[^0-9]', ''))/1000))

cols = ['ProductOffers','TimeOfOfferChange']
offerData = reduce(lambda x,y: x.drop(y),cols,res)
offerData.show(1)

SyntaxError: invalid syntax (<ipython-input-12-d41651559083>, line 5)

In [ ]:
%pyspark


offerData.write.csv("s3a://offer-flatten/output321/", sep=',')

In [6]:
from datetime import datetime 

TC_Preditor_Name = "TC_Preditor_{0}.csv".format(datetime.utcnow().strftime("utc%Y%m%d"))
TC_Preditor_Name_prefixKey =  (datetime.utcnow()).strftime("prod/trialconversion-predictor/minPriceDatapoints/%Y/%m/%d/") + TC_Preditor_Name

#offerData.write.csv("s3a://offer-flatten/FlattenOutput/{0}".format(TC_Preditor_Name_prefixKey), sep=',')

In [7]:
TC_Preditor_Name_prefixKey

'prod/trialconversion-predictor/minPriceDatapoints/2017/10/02/TC_Preditor_utc20171002.csv'

In [9]:
a = ['US','MO']

In [11]:
a[0]+'-'+a[1]

'US-MO'

In [13]:
#lambda x:x[0]+'-'+x[1]


#%pyspark


res = offerDF.withColumn("ItemCondition", col("ProductOffers.Identifier.ItemCondition"))\
.withColumn("IsBuyBoxWinner", explode(col("ProductOffers.Offers.IsBuyBoxWinner")))\
.withColumn("IsExpeditedShippingAvailable", explode(col("ProductOffers.Offers.IsExpeditedShippingAvailable")))\
.withColumn("IsFeaturedMerchant", explode(col("ProductOffers.Offers.IsFeaturedMerchant")))\
.withColumn("listingprice_amount", explode(col("ProductOffers.Offers.ListingPrice.Amount")))\
.withColumn("listingprice_currency", explode(col("ProductOffers.Offers.ListingPrice.CurrencyCode")))\

.withColumn("availabilityType", explode(col("ProductOffers.Offers.ShippingTime.AvailabilityType")))\
.withColumn("availableDate", explode(col("ProductOffers.Offers.ShippingTime.AvailableDate")))\
.withColumn("maximumHours", explode(col("ProductOffers.Offers.ShippingTime.MaximumHours")))\
.withColumn("minimumHours", explode(col("ProductOffers.Offers.ShippingTime.MinimumHours")))\
.withColumn("shipsDomestically", explode(col("ProductOffers.Offers.ShipsDomestically")))\

#.withColumn("shipsFrom", explode(col("ProductOffers.Offers.ShipsFrom")  ))\
.withColumn("shipsFromState", explode(col("ProductOffers.Offers.ShipsFrom.State")))\
.withColumn("shipsFromCountry", explode(col("ProductOffers.Offers.ShipsFrom.Country")))\

.withColumn("shipsInternationally", explode(col("ProductOffers.Offers.ShipsInternationally")))

cols = ['ProductOffers','TimeOfOfferChange']
offerData = reduce(lambda x,y: x.drop(y),cols,res)
offerData.show(1)

SyntaxError: invalid syntax (<ipython-input-13-6f391ebefc07>, line 8)

In [ ]:
335,544,32 rows,  5 GB,  13.5 MINs

In [ ]:
%pyspark

#Flatten for offers,  262,144 rows, take 19s save to s3

Offers = offerDF.withColumn("ItemCondition", col("ProductOffers.Identifier.ItemCondition"))\
.withColumn("IsBuyBoxWinner", explode(col("ProductOffers.Offers.IsBuyBoxWinner")))\
.withColumn("IsFeaturedMerchant", explode(col("ProductOffers.Offers.IsFeaturedMerchant")))\
.withColumn("listingprice_amount", explode(col("ProductOffers.Offers.ListingPrice.Amount")))\
.withColumn("listingprice_currency", explode(col("ProductOffers.Offers.ListingPrice.CurrencyCode")))\
.withColumn("subCondition", explode(col("ProductOffers.Offers.SubCondition")))\
.withColumn("Points", explode(col("ProductOffers.Offers.Points")))\
.withColumn("FeedbackCount", explode(col("ProductOffers.Offers.SellerFeedbackRating.FeedbackCount")))\
.withColumn("SellerPositiveFeedbackRating", explode(col("ProductOffers.Offers.SellerFeedbackRating.SellerPositiveFeedbackRating")))\
.withColumn("SellerId", explode(col("ProductOffers.Offers.SellerId")))


cols = ['ProductOffers','TimeOfOfferChange']
offersData = reduce(lambda x,y: x.drop(y),cols,Offers)
#offersData.show(1)
offersData.count()

In [ ]:
%pyspark
#Flatten for offer Shipping, 419,4304 rows, take 50s save to s3

OfferShippings = offerDF.withColumn("shipping_amount", explode(col("ProductOffers.Offers.Shipping.Amount")))\
.withColumn("shipping_currency", explode(col("ProductOffers.Offers.Shipping.CurrencyCode")))\
.withColumn("shipsInternationally", explode(col("ProductOffers.Offers.ShipsInternationally")))\
.withColumn("IsExpeditedShippingAvailable", explode(col("ProductOffers.Offers.IsExpeditedShippingAvailable")))\
.withColumn("availabilityType", explode(col("ProductOffers.Offers.ShippingTime.AvailabilityType")))\
.withColumn("availableDate", explode(col("ProductOffers.Offers.ShippingTime.AvailableDate")))\
.withColumn("maximumHours", explode(col("ProductOffers.Offers.ShippingTime.MaximumHours")))\
.withColumn("minimumHours", explode(col("ProductOffers.Offers.ShippingTime.MinimumHours")))\
.withColumn("shipsDomestically", explode(col("ProductOffers.Offers.ShipsDomestically")))\
.withColumn("shipsFromState", explode(col("ProductOffers.Offers.ShipsFrom.State")))\
.withColumn("shipsFromCountry", explode(col("ProductOffers.Offers.ShipsFrom.Country")))



cols = ['ProductOffers','TimeOfOfferChange']
OfferShippingData = reduce(lambda x,y: x.drop(y),cols,OfferShippings)
#OfferShippingData.show(1)
OfferShippingData.count()


In [ ]:
#Partition the output and save to s3
#2 min 2 sec
newOfferShippingData = OfferShippingData.repartition(100)
newOfferShippingData.rdd.getNumPartitions()

OfferSummary_prefixKey =  (datetime.utcnow()).strftime("s3a://offer-flatten/offer-summary-partition/%Y/%m/%d/")
newOfferShippingData.write.csv(OfferSummary_prefixKey, sep=',')

In [ ]:
# Partition by ItemId, 46 sec.
OfferShipping_prefixKey =  (datetime.utcnow()).strftime("s3a://offer-flatten/offer-summary-partition2/%Y/%m/%d/")
OfferShippingData.repartition("ItemId").write.partitionBy("ItemId").csv(OfferShipping_prefixKey, sep=',')

In [ ]:
#Aother way to partition
df.write.partitionBy(['inserted_at_day']).format("parquet").save(dest, mode="append")

In [ ]:
%pyspark
#Flatten for offer summary, 128 rows, take 


OfferSummary = offerDF.withColumn("BuyBoxEligibleOffersCondition", explode(col("ProductOffers.Summary.BuyBoxEligibleOffers.Condition")))\
.withColumn("FulfillmentChannel", explode(col("ProductOffers.Summary.BuyBoxEligibleOffers.FulfillmentChannel")))\
.withColumn("BuyBoxEligibleOffersCount", explode(col("ProductOffers.Summary.BuyBoxEligibleOffers.Value")))\
.withColumn("LandedPrice_Amount", explode(col("ProductOffers.Summary.BuyBoxPrices.LandedPrice.Amount")))\
.withColumn("LandedPrice_Currency", explode(col("ProductOffers.Summary.BuyBoxPrices.LandedPrice.CurrencyCode")))\
.withColumn("BB_listPrice_Amount", explode(col("ProductOffers.Summary.BuyBoxPrices.ListingPrice.Amount")))\
.withColumn("BB_ListPrice_Currency", explode(col("ProductOffers.Summary.BuyBoxPrices.ListingPrice.CurrencyCode")))\
.withColumn("ListPrice_Amount", col("ProductOffers.Summary.ListPrice.Amount"))\
.withColumn("ListPrice_Currency", col("ProductOffers.Summary.ListPrice.CurrencyCode"))\
.withColumn("ListingPrice", explode(col("ProductOffers.Summary.LowestPrices.ListingPrice.Amount")))\
.withColumn("ListingPrice_Currency", explode(col("ProductOffers.Summary.LowestPrices.ListingPrice.CurrencyCode")))\
.withColumn("MinimumAdvertisedPrice", col("ProductOffers.Summary.MinimumAdvertisedPrice"))\
.withColumn("SalesRankings", explode(col("ProductOffers.Summary.SalesRankings.ProductCategoryId")))\
.withColumn("NumberOfOffers", explode(col("ProductOffers.Summary.SalesRankings.Rank")))\
.withColumn("SuggestedLowerPricePlusShipping",col("ProductOffers.Summary.SuggestedLowerPricePlusShipping"))\
.withColumn("TotalOfferCount",col("ProductOffers.Summary.TotalOfferCount"))\
.withColumn('OfOfferChangeDatetime', from_unixtime((regexp_replace('TimeOfOfferChange','[^0-9]', ''))/1000))


cols = ['ProductOffers','TimeOfOfferChange']
OfferSummaryData = reduce(lambda x,y: x.drop(y),cols,OfferSummary)
#OfferSummaryData.show(1)
OfferSummaryData.count()